In [2]:
import numpy as np
import random
import copy
import gymnasium as gym
from evogym.envs import *
from evogym import EvoWorld, EvoSim, EvoViewer, sample_robot, get_full_connectivity, is_connected
import utils
from fixed_controllers import *

# Imports to compare performances
import time
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def run_experiment(algorithm, num_runs=1, num_generations=100, scenario='Walker-v0', steps=500, controller=None):
    """
    Run an evolutionary algorithm multiple times and store key results.
    
    Parameters:
        algorithm (function): The function that runs an evolutionary algorithm (e.g., `random_search`, `differential_evolution`).
        num_runs (int): Number of times to run the algorithm.
        num_generations (int): Number of generations per run.
        scenario (str): Environment scenario.
        steps (int): Number of steps per simulation.
        controller (function): Robot controller function.

    Returns:
        best_overall_robot (EvoRobot): Best robot found across all runs.
        best_overall_fitness (float): Best fitness score found across all runs.
        mean_fitness_per_generation (np.array): Mean best fitness per generation across all runs.
        mean_fitnesses (np.array): Mean of mean fitness scores per generation.
        mean_execution_time (float): Mean execution time across all runs.
        std_fitnesses (np.array): Standard deviation of fitness scores per generation.
    """

    # Initialize storage variables
    best_fitnesses_overall = np.zeros(num_generations)  # Sum fitness scores across runs (to average later)
    mean_fitnesses = np.zeros(num_generations)  # Sum mean fitness scores across runs (to average later)
    std_fitnesses = np.zeros(num_generations)  # Sum std fitness scores across runs (to average later)
    
    total_execution_time = 0
    best_overall_fitness = float('-inf')
    best_overall_robot = None

    for i in range(num_runs):
        print(f"\nRunning {algorithm.__name__} {i + 1}/{num_runs}...")

        # Run the algorithm and extract results
        best_robot, best_fitness, best_fitness_scores, mean_fitnesses_run, execution_time, std_fitness_scores_run = algorithm()

        print(f"Best fitness score of run {i + 1}: {best_fitness:.3f}")

        # Accumulate fitness data for averaging
        best_fitnesses_overall += np.array(best_fitness_scores)  
        mean_fitnesses += np.array(mean_fitnesses_run)  
        std_fitnesses += np.array(std_fitness_scores_run)

        # Keep track of the best robot across all runs
        if best_fitness > best_overall_fitness:
            best_overall_fitness = best_fitness
            best_overall_robot = best_robot

        # Accumulate execution time
        total_execution_time += execution_time

    # Compute the **mean** best fitness per generation across runs
    best_fitnesses_overall /= num_runs
    mean_fitnesses /= num_runs
    std_fitnesses /= num_runs
    mean_execution_time = total_execution_time / num_runs

    print("\nFinal Results After Multiple Runs:")
    print(f"Mean execution time: {mean_execution_time:.2f} seconds")
    print(f"Best fitness found: {best_overall_fitness:.3f}")

    # Plot averaged fitness evolution
    plt.figure(figsize=(10, 5))
    plt.plot(range(num_generations), best_fitnesses_overall, label="Best Fitness per Generation", color='blue')
    plt.fill_between(range(num_generations), best_fitnesses_overall - std_fitnesses, best_fitnesses_overall + std_fitnesses, color='blue', alpha=0.2, label="Std Dev Range")
    plt.xlabel("Generation")
    plt.ylabel("Fitness Score")
    plt.title(f"{algorithm.__name__}: Fitness Evolution in {scenario}")
    plt.legend()
    plt.grid()
    plt.show()

    # Simulate best robot
    print("\nSimulating Best Robot...")
    for _ in range(10):
        utils.simulate_best_robot(best_overall_robot, scenario=scenario, steps=steps)

    # Create a visualization GIF
    utils.create_gif(best_overall_robot, filename=f'{algorithm.__name__}_{scenario}_high_gen_structure.gif', scenario=scenario, steps=steps, controller=controller)

    # Return results
    return best_overall_robot, best_overall_fitness, best_fitnesses_overall, mean_fitnesses, mean_execution_time, std_fitnesses


In [ ]:

# ---- PARAMETERS ----
POP_SIZE = 40        # Population size
NUM_GENERATIONS = 50  # Number of generations
MUTATION_RATE = 0.1  # Probability of mutation per voxel
ELITE_RATIO = 0.2    # Percentage of best individuals to keep
STEPS = 500
SCENARIO = 'Walker-v0'

# ---- VOXEL TYPES ----
VOXEL_TYPES = [0, 1, 2, 3, 4]  # Empty, Rigid, Soft, Active (+/-)

# CONTROLLER = alternating_gait  # Fixed controller
# CONTROLLER = sinusoidal_wave   # Fixed controller
CONTROLLER = hopping_motion    # Fixed controller

def evaluate_fitness(robot_structure, view=False):
    """Evaluate the fitness of a given robot structure."""
    try:
        connectivity = get_full_connectivity(robot_structure)
        env = gym.make(SCENARIO, max_episode_steps=STEPS, body=robot_structure, connections=connectivity)
        env.reset()
        sim = env.sim
        viewer = EvoViewer(sim)
        viewer.track_objects('robot')
        t_reward = 0
        action_size = sim.get_dim_action_space('robot') # Get correct action size

        for t in range(STEPS):
            # Update actuation before stepping
            actuation = CONTROLLER(action_size, t)
            if view:
                viewer.render('screen')
            ob, reward, terminated, truncated, info = env.step(actuation)
            t_reward += reward

            if terminated or truncated:
                break
            
        viewer.close()
        env.close()
        return t_reward
    except:
        return 0.0  # Invalid robots get zero fitness

def create_random_robot():
    """Generate a random valid robot structure."""
    grid_size = (5, 5)
    robot, _ = sample_robot(grid_size)
    return robot

# This may be the reason why the code is so slow
def mutate(robot):
    """Mutate a robot's structure by randomly changing voxels."""
    """Point mutation"""
    new_robot = copy.deepcopy(robot)
    for i in range(new_robot.shape[0]):
        for j in range(new_robot.shape[1]):
            if random.random() < MUTATION_RATE: # We have a 10% chance of mutation
                new_robot[i, j] = random.choice(VOXEL_TYPES)
    return new_robot if is_connected(new_robot) else robot  # Ensure connectivity, otherwise return original

# This may be the reason why the code is so slow
def crossover(parent1, parent2):
    """Perform crossover between two parent robots."""
    """Uniform crossover"""
    child = copy.deepcopy(parent1)
    for i in range(child.shape[0]):
        for j in range(child.shape[1]):
            if random.random() < 0.5: # 50% chance of inheriting from parent2
                child[i, j] = parent2[i, j]
    return child if is_connected(child) else parent1  # Ensure connectivity

def select_parents(population, fitnesses):
    """Select parents using tournament selection."""
    """Tournament selection"""
    tournament_size = 3
    parents = []
    for _ in range(len(population)):
        competitors = random.sample(list(zip(population, fitnesses)), tournament_size)
        best = max(competitors, key=lambda x: x[1])[0]
        parents.append(best)
    return parents

def genetic_algorithm():
    """Run the evolutionary algorithm to evolve the best robot structure."""
    # Initialize population
    population = [create_random_robot() for _ in range(POP_SIZE)]
    best_fitness_scores = []
    mean_fitness_scores = []
    std_fitness_scores = []
    
    start_time = time.time()  # Start measuring time

    for generation in range(NUM_GENERATIONS):
        # Evaluate fitness
        fitnesses = [evaluate_fitness(robot) for robot in population]
        best_idx = np.argmax(fitnesses)
        
        # Record fitness values for plotting
        best_fitness_scores.append(fitnesses[best_idx])
        mean_fitness_scores.append(np.mean(fitnesses))
        std_fitness_scores.append(np.std(fitnesses))

        # Report progress
        # print(f"Generation {generation + 1}: Best Fitness = {fitnesses[best_idx]:.3f}, Mean Fitness = {np.mean(fitnesses):.3f}")

        # Select top individuals (elitism)
        num_elites = int(ELITE_RATIO * POP_SIZE)
        elites = [population[i] for i in np.argsort(fitnesses)[-num_elites:]]

        # Select parents and create offspring
        parents = select_parents(population, fitnesses)
        offspring = [crossover(random.choice(parents), random.choice(parents)) for _ in range(POP_SIZE - num_elites)]
        
        # Apply mutation
        offspring = [mutate(child) for child in offspring]

        # Form the new population
        population = elites + offspring
    
    end_time = time.time()  # End time measurement
    execution_time = end_time - start_time

    return population[np.argmax(fitnesses)], max(fitnesses), best_fitness_scores, mean_fitness_scores, execution_time, std_fitness_scores


In [ ]:
# Run the genetic algorithm
best_robot_genetic, best_fitness_genetic, best_fitnesses_genetic, mean_fitnesses_genetic, execution_time_genetic, std_fitness_scores_genetic = run_experiment(genetic_algorithm, num_generations=NUM_GENERATIONS, scenario=SCENARIO, steps=STEPS, controller=CONTROLLER)



Running genetic_algorithm 1/1...
SIMULATION UNSTABLE... TERMINATING
